In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
pd.options.mode.chained_assignment = None 


In [ ]:
# read the data
df_train = pd.read_csv('train_small.csv')
df_train.head()


# EDA

In [ ]:
df_train.info()


In [ ]:
df_train['is_brand'].unique()

In [ ]:
df_train['competitor1_rate'].unique()
# integer without nans


In [ ]:
df_train['competitor1_has_availability'].unique()
# integer without nans


In [ ]:
df_train['competitor1_price_percent_diff'].unique()
# float


In [ ]:
# correct the types
df_train['stay_on_saturday'] = df_train['stay_on_saturday'].astype('bool')
df_train['random_sort'] = df_train['random_sort'].astype('bool')
df_train['clicked'] = df_train['clicked'].astype('bool')
df_train['booked'] = df_train['booked'].astype('bool')
df_train['is_brand'] = df_train['is_brand'].astype('bool')
df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
# only change price percent diff
cols_comp = [col for col in df_train.columns if 'price_percent_diff' in col]
df_train[cols_comp] = df_train[cols_comp].astype('float')

df_train.info()


## Q.1/a

In [ ]:
df_train.dtypes.to_excel('dtypes.xlsx')

In [ ]:
df_train.timestamp.sort_values()

In [ ]:
print('There are %d datapoints and %d columns.'%(df_train.shape[0], df_train.shape[1]))
print('There are %d unique searches.'%len(df_train['search_id'].unique()))
print('The data is shows the searches done within less than 1 year.')


In [ ]:
df_train['search_year'] = pd.DatetimeIndex(df_train['timestamp']).year
df_train['search_month'] = pd.DatetimeIndex(df_train['timestamp']).month
df_train['search_dayofweek'] = pd.DatetimeIndex(df_train['timestamp']).dayofweek
df_train['search_dayofyear'] = pd.DatetimeIndex(df_train['timestamp']).dayofyear
df_train['search_hour'] = pd.DatetimeIndex(df_train['timestamp']).hour
df_train['search_quarter'] = pd.DatetimeIndex(df_train['timestamp']).quarter

# for deeper analysis:
# some holidays, business day?, business hour?, etc


In [ ]:
# are there any duplicate records?
df_train[df_train.duplicated()]

In [ ]:
# are there any missing values?
print('There are %d missing values.'%df_train.isna().sum().sum())
print()
ds = df_train.isna().sum()
cols_missing = ds[ds != 0].sort_values(ascending=False)/len(df_train)*100

print(cols_missing)


In [ ]:
# summary statistics
df_train.describe()

In [ ]:
df_train.describe(include=['bool'])

In [ ]:
# more than one click in the search?

df_ = df_train.groupby('search_id')['clicked'].sum().sort_values()
print(df_.value_counts(normalize=True))
df_


In [ ]:
df_ = df_train[['listing_id', 'clicked', 'booked']]
df_['clicked'].value_counts(normalize=True)


In [ ]:
df_[df_['clicked'] == 1]['booked'].value_counts(normalize=True)

In [ ]:
# CTR/CR

# so slow running, can do better

df_r = pd.DataFrame()

counts = []
lists = []
clicks = []
books = []

for l in df_train['listing_id'].unique():
    df_ = df_train[df_train['listing_id'] == l]
    counts.append(len(df_))
    lists.append(l)
    clicks.append(len(df_[df_['clicked'] == 1]))
    books.append(len(df_[df_['booked'] == 1]))
   
df_r['listing_id'] = lists
df_r['counts'] = counts
df_r['books'] = books
df_r['clicks'] = clicks
df_r['click_thru_rate'] = df_r['clicks'] / df_r['counts']
df_r['conversion_rate'] = df_r['books'] / df_r['counts']
df_r['listing_review_score'] = df_train['listing_review_score'] 
df_r['listing_stars'] = df_train['listing_stars']


In [ ]:
print('%d listings have 100%% click through rate.'%len(df_r[df_r['click_thru_rate'] == 1])
)

print('%d listings have 100%% conversion rate.'%len(df_r[df_r['conversion_rate'] == 1])
)

print(len(df_r[df_r['conversion_rate'] == 1])/len(df_train['listing_id'].unique()))


In [ ]:
df_r.corr()

In [ ]:
searches = df_train['search_id'].unique()
'There are %d searches'%len(searches)


# <html><p style='color: #FF3341'> add</p></html>

In [ ]:
# how many of the searched ended up with a booking





In [ ]:
# user_country_id versus listing_country_id
print('Have users from %d different countries.'%len(df_train['user_country_id'].unique()))
print('Have listings in %d different countries.'%len(df_train['listing_country_id'].unique()))


In [ ]:
# for each search how many datapoints and how many unique listings
is_repeat = 0

for s in searches:
    
    df_ = df_train[df_train['search_id'] == s]
    len_lists = len(df_)
    len_lists_unique = len(df_['listing_id'].unique())
    
    if(len_lists_unique != len_lists):
        print(s, 'A listing is displayed more than once.')
        is_repeat = 1

if(~is_repeat):
    print('Within each search, there is only one listing per hotel.')
    

In [ ]:
# site_id
len(df_train['site_id'].unique())

In [ ]:
# check if there is any customer who rated the hotel
# but have no purchases for that hotel

si = df_train[(df_train['user_hist_paid'].isna()) & (~df_train['user_hist_stars'].isna())]['search_id'].unique()
print('In the search %d, the user hasn\'t purchased \
anything on the website but there is a rating by the customer.\
 There should be a mistake in the dataset, hence I will \
discard the records for search %d.'%(si, si))

df_train = df_train[~df_train['search_id'].isin(si)]


In [ ]:
# first time customer ratio
df_ = df_train[['search_id', 'user_hist_paid']].drop_duplicates()
ftr = len(df_[df_['user_hist_paid'].isna()])/len(df_)*100
print('%d%% of the searches are done by a first time customer.\n'%ftr)
# if that is unreliable, use this

df_ = df_train[['search_id', 'user_country_id', 'user_hist_stars', 'user_hist_paid']]
len_all = len(df_)
df_ = df_[~df_.isna().any(axis=1)]
df_['user_id'] = df_['user_country_id'].astype('str') + '|' + df_['user_hist_stars'].astype('str') + '|' + df_['user_hist_paid'].astype('str')
len_not_first = len(df_[~df_['user_hist_paid'].isna()]['user_id'].unique())
first_time_ratio = (1 - len_not_first/len_all)*100

print('There is no user ID on the data to say whether \
a user is a first-time customer. The user_hist_stars \
and user_hist_paid values migth be unreliable.\
 Therefore, we can create a user ID using user-related \
data to give a rough conlusion. Discarding null data, \
there are supposedly %d users \
with %d different related searches. This gives that \
roughly %.2f percent of the\
 searches are done by a first-time customer.\
'%(len_not_first, len(df_['search_id'].unique()), first_time_ratio))

print('\nCombining these two methods, we can say that \
the majority of the searches are done by a \
first-time customer (a customer who has never \
booked a hotel on the website) or an anonymous user.')


In [ ]:
# listings
hotels = df_train['listing_id'].unique()
countries = df_train['listing_country_id'].unique()

print('There are %d hotels listed in the data, located in %d different countries\
.\n'%(len(hotels), len(countries)))

# pie chart
df_ = df_train['listing_country_id'].value_counts().to_frame()
quantile = df_.quantile(0.97).iloc[0]
df_q = df_[df_['listing_country_id'] > quantile]
others = df_[df_['listing_country_id'] <= quantile]['listing_country_id'].sum()
new_row = pd.DataFrame([others], columns=['listing_country_id'], index=['others'])
df_q = pd.concat([df_q, new_row])

print('\nPie chart showing top %d countries.'%len(df_q))

df_q.plot.pie(y='listing_country_id')
plt.show()

# how many hotels has ratings
len_rated = len(df_train[df_train['listing_stars'] != 0]['listing_id'].unique())
len_reviewed = len(df_train[df_train['listing_review_score'] != 0]['listing_id'].unique())

df_both = df_train[(df_train['listing_review_score'] != 0) & 
               (df_train['listing_stars'] != 0)]['listing_id'].unique()


print('Roughly %.2f%% of the hotels are rated and %.2f%% are reviewed,\
 only %.2f%% have both ratings and reviews.\
 \n'%(len_rated/len(hotels)*100, len_reviewed/len(hotels)*100, 
      len(df_both)/len(hotels)*100))

# hotel stars and reviews- compare
df_ = df_train[(df_train['listing_stars'] != 0) & (df_train['listing_review_score'] != 0)][['listing_stars', 'listing_review_score']]
df_['score_diff'] = abs(df_['listing_stars'] - df_['listing_review_score'])
df_.boxplot()
plt.title('Rating and review score differences (both rated&reviewed)')
plt.show()

In [ ]:
# how many time each hotel listed
df_train['listing_id'].value_counts().to_frame()

In [ ]:
# location_score1 and location_score2, which of them affects the booking ratio more
# hotel stars and reviews- compare
df_ = df_train[['location_score1', 'location_score2']]

# how many hotels has location scores
len_score1 = len(df_train[~df_train['location_score1'].isna()]['listing_id'].unique())
len_score2 = len(df_train[~df_train['location_score2'].isna()]['listing_id'].unique())

df_both = df_train[(~df_train['location_score1'].isna()) & 
               (~df_train['location_score2'].isna())]['listing_id'].unique()


print('%.2f%% of the hotels has location_score1 and %.2f%% location_score_2,\
 only %.2f%% have both.\
 \n'%(len_score1/len(hotels)*100, len_score2/len(hotels)*100, 
      len(df_both)/len(hotels)*100))

print(df_.describe())
df_.boxplot()
plt.title('Location score differences')
plt.show()

In [ ]:
# what is log_historical_price exactly?
print(df_train['log_historical_price'].describe())

# case 1: listing_id = 3625
h_ = 3625

df_ = df_train[(df_train['listing_id'] == h_) & (df_train['log_historical_price'] != 0)]
prices = df_['log_historical_price'].unique()
day_range = (df_['timestamp'].max() - df_['timestamp'].min()).days

print('Analysis of a hotel\'s historical prices:\n\
The hotel %d has had these prices over the periods:\n\
'%h_, prices)

print('\nSince the data for hotel %d spans over %d days,\
 there is a low chance that the hotel was sold %d times \
during that period.'%(h_, day_range, len(prices)))


In [ ]:
# see how many times each hotel was sold
df_times_sold = pd.DataFrame()
t = []
for h in hotels:
    t.append(len(df_train[(df_train['listing_id'] == h) & (df_train['log_historical_price'] != 0)]['log_historical_price'].unique()))
  
df_times_sold['hotel'] = hotels
df_times_sold['times_sold'] = t

df_times_sold.sort_values(by=['times_sold'], ascending=False)


In [ ]:
# case 2: listing_id = 30992
h_ = 30992

df_ = df_train[(df_train['listing_id'] == h_) & (df_train['log_historical_price'] != 0)]
prices = df_['log_historical_price'].unique()
day_range = (df_['timestamp'].max() - df_['timestamp'].min()).days

print('Analysis of a hotel\'s historical prices:\n\
The hotel %d has had these prices over the periods:\n\
'%h_, prices)

print('\nSince the data for hotel %d spans over %d days,\
 there is a low chance that the hotel was sold %d times\
 during that period.'%(h_, day_range, len(prices)))


In [ ]:
'From this analysis, I conclude that historical price \
means something else. It might be the historical prices\
 of the listings, which makes more sense in this way.\
 From now on, I will treat it like so.'


In [ ]:
# add a feature: how many a times a listing is sold
print(df_train.shape)
df_train = df_train.merge(df_times_sold, how='left', left_on='listing_id', right_on='hotel').drop(['hotel'], axis=1)
print(df_train.shape)


In [ ]:
# how many listings listed in each search result
for s in searches:
    df_ = df_train[df_train['search_id'] == s]
    n_unique_listing = len(df_['listing_id'].unique())
    n_unique_position = len(df_['listing_position'].unique())
    if(n_unique_listing != n_unique_position):
        print('Not match: %d'%s)
        
df_train.groupby('search_id')['listing_position'].count().hist(figsize=(10, 5))
print('How many listings in each search result:')
print(df_train.groupby('search_id')['listing_position'].count())
plt.show()


In [ ]:
# does listing_position affect the click/purchase?
df_ = df_train[['listing_position', 'clicked', 'booked']]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
lim1 = df_[df_['booked'] == 1]['listing_position'].hist(ax=axes[0]).get_ylim()[1]
lim2 = df_[df_['clicked'] == 1]['listing_position'].hist(ax=axes[1]).get_ylim()[1]

axis_lim = max(lim1, lim2)
axes[0].set_ylim(0, axis_lim)
axes[1].set_ylim(0, axis_lim)

axes[0].set_title('booked')
axes[1].set_title('clicked')
print(df_.describe())
print('\nHow the listing position affects booking and clicking')
plt.show()


In [ ]:
df_['first_position'] = df_['listing_position'].apply(lambda x: 1 if x == 1 else 0)
len_b = df_[df_['first_position'] == 1]['booked'].sum()
len_c = df_[df_['first_position'] == 1]['clicked'].sum()
len_first = len(df_[df_['first_position'] == 1])

print('%.2f %% of the listings in the first position are clicked, \
%.2f %% booked.'%(len_c/len_first*100, len_b/len_first*100))


In [ ]:
# price of the hotel
print(df_train['price_usd'].describe())


In [ ]:
# has_promotion
print(df_train['has_promotion'].unique())

df_train[['has_promotion', 'clicked', 'booked']].corr()


In [ ]:
# booking_value vs price_usd

#discrepancy?
l = len(df_train[(df_train['booked'] == 0) & (~df_train['booking_value'].isna())])
print('There are %d record showing that booking value exists\
given that the listing is booked.\n'%l)

df_ = df_train[df_train['booked'] == 1][['booking_value', 'price_usd']]
df_['price_diff'] = df_train['booking_value'] - df_train['price_usd']
print(df_['price_diff'].describe())


In [ ]:
# destination_id, for each searh how many listing_country_id
print('There are %d destinations. Assuming, these destinations\
are the locations where the user want to book a hotel room in,\
 I already checked if each search result has the same destination ID.\
 '%len(set(df_train['destination_id'])))
df_ = df_train[['booked', 'destination_id']].drop_duplicates()

df_['destination_id'].hist()
plt.title('distribution of destinations searched')
plt.show()

print('The data seems evenly distributed over destionations.')

In [ ]:
df_ = df_train.groupby('destination_id').sum()['times_sold'].to_frame().reset_index()
print(df_['times_sold'].describe())
print(df_['times_sold'].plot())
#quantile = df_[df_['times_sold'] != 0]['times_sold'].quantile(0.99)
#df_q = df_[df_['times_sold'] > quantile]

df_ = df_.sort_values(by='times_sold', ascending=False)
print(df_['times_sold'].iloc[0]/df_['times_sold'].sum())
print(len(df_train[df_train['destination_id'] == 8192][['search_id', 'destination_id']].drop_duplicates())/len(df_train['search_id'].unique()))
df_

In [ ]:
# booking_window statistics
df_ = df_train[['search_id', 'booking_window']].drop_duplicates()
print(df_['booking_window'].value_counts())
print('\n%.2f%% of the booking windows searched is 1 day.\
'%(df_['booking_window'].value_counts().head().iloc[0]/len(df_)*100))
df_['booking_window'].hist()
plt.show()


In [ ]:
print('\n%.2f%% of searches has no booking window.\
'%(df_['booking_window'].value_counts().head().iloc[1]/len(df_)*100))


In [ ]:
print('\n%.2f%% of searches has 2-day booking window.\
'%(df_['booking_window'].value_counts().head().iloc[2]/len(df_)*100))


In [ ]:
# num_adults/num_kids/num_rooms/length_of_stay
# are these the same for each search?

print('Since there is only one value for num_adults/num_kids/\
num_rooms/length_of_stay for each search, these are specified \
by the user.\n')

df_ = df_train[['search_id', 'num_adults', 'num_kids', 'num_rooms', 'length_of_stay']]
df_.drop_duplicates(inplace=True)

print('Different lengths of stays:\n')
length_of_stays = df_['length_of_stay'].sort_values().unique()
print(length_of_stays)
print()
df_['length_of_stay'].hist(figsize=(10, 5), bins=length_of_stays)
plt.title('Lenth_of_stay distribution')
plt.show()

print('57 seems odd so let\'s check it out:\n')

print(df_train[df_train['length_of_stay'] == 57][['clicked', 'booked']])


df_['length_of_stay'].value_counts(normalize=True)


In [ ]:
# how many 1-day stays booked?

df_ = df_train[df_train['length_of_stay'] == 1][['search_id', 'booked']]
df_['booked'] = df_['booked'].apply(lambda x: 1 if x == True else 0)
df_.groupby('search_id')['booked'].sum().value_counts(normalize=True)

# more:
# look at their booking window


In [ ]:
# stay_on_saturday, ratio of searches including saturday?
df_ = df_train[['search_id', 'length_of_stay','stay_on_saturday']]
df_.drop_duplicates(inplace=True)

print('If the stay lasts more than 6 days then it \
should include at least 1 Saturday.\n')

print('Different lengths of stay with Saturday:\n')

print(df_[df_['stay_on_saturday'] == True]['length_of_stay'].unique())

print('\nDifferent lengths of stay with no Saturday:\n')

print(df_[df_['stay_on_saturday'] == False]['length_of_stay'].unique())

print('\nHow come a stay lasts more than 7 days and \
still has no Saturday stay?')

In [ ]:
for i in range(len(df_)):
    is_sat = 1 if df_['stay_on_saturday'].iloc[i] == True else 0
    times = df_['length_of_stay'].iloc[i] // 7
    
    if((is_sat == 0) & (times >= 1)):
        print('Search %d has a problem.'%df_['search_id'].iloc[i])
        break
print()
print(df_[df_['search_id'] == 30])

print('\n\nLet\'s look at search 30:\n')
print('Date: %s\nWindow: %d\nLength: %d\nSaturday: %s'\
%(df_train[df_train['search_id'] == 30]['timestamp'].iloc[0],
df_train[df_train['search_id'] == 30]['booking_window'].iloc[0],
df_train[df_train['search_id'] == 30]['length_of_stay'].iloc[0],
df_train[df_train['search_id'] == 30]['stay_on_saturday'].iloc[0],
))

print('\nThis stay seems to include a Saturday (2/3/201) \
and it starts on Sunday.')


In [ ]:
# Let's include the start date of the stay
df_train['stay_start'] = df_train['timestamp'] + df_train['length_of_stay'].apply(timedelta)

df_train['stay_year'] = pd.DatetimeIndex(df_train['stay_start']).year
df_train['stay_month'] = pd.DatetimeIndex(df_train['stay_start']).month
df_train['stay_dayofweek'] = pd.DatetimeIndex(df_train['stay_start']).dayofweek
df_train['stay_dayofyear'] = pd.DatetimeIndex(df_train['stay_start']).dayofyear
df_train['stay_hour'] = pd.DatetimeIndex(df_train['stay_start']).hour
df_train['stay_quarter'] = pd.DatetimeIndex(df_train['stay_start']).quarter


In [ ]:
print('stay start days with saturday:', df_train[df_train['stay_on_saturday'] == True]['stay_dayofweek'].unique()
)
print('\nI couldn\'t understand this feature so I will continue \
without more analysis.')

In [ ]:
df_ = df_train[['search_id', 'destination_id', 'stay_dayofyear', 'stay_month', 'stay_dayofweek']]
df_.drop_duplicates(inplace=True)
print(df_['destination_id'].value_counts(ascending=False, normalize=True).head())
print('\n')
print(df_['stay_dayofyear'].value_counts(ascending=False, normalize=True))
print('\n')
print(df_['stay_month'].value_counts(ascending=False, normalize=True))
print('\n')
print(df_['stay_dayofweek'].value_counts(ascending=False, normalize=True))



In [ ]:
len(df_[df_['stay_month'].isin([6, 3, 5, 4])])/len(df_)

In [ ]:
# distance_to_dest
df_train['distance_to_dest'].describe()


In [ ]:
# a listing is in more than one site?


In [ ]:
# how many of the clicked ones booked?


In [ ]:
df_train.info()

In [ ]:
# fill nulls


# listing_review_score:
# average
print(len(df_train['listing_id'].unique()))

df_ = df_train[['listing_id', 'listing_review_score']].drop_duplicates()
score_avg = df_['listing_review_score'].mean()
print(df_.shape)
df_train['listing_review_score'] = df_train['listing_review_score'].fillna(score_avg)


# distance_to_dest:
# average
print(len(df_train['listing_id'].unique()))

df_ = df_train[['search_id', 'distance_to_dest']].drop_duplicates()
dist_avg = df_['distance_to_dest'].mean()
print(df_.shape)
df_train['distance_to_dest'] = df_train['distance_to_dest'].fillna(dist_avg)

df_train

In [ ]:
# competitor info
df_train = df_train.rename(columns={'competitor1_rate':'cr1',
                 'competitor1_has_availability':'cha1', 
                 'competitor1_price_percent_diff':'cppf1',
                 'competitor2_rate':'cr2', 
                 'competitor2_has_availability':'cha2',
                 'competitor2_price_percent_diff':'cppf2', 
                 'competitor3_rate':'cr3',
                 'competitor3_has_availability':'cha3', 
                 'competitor3_price_percent_diff':'cppf3',
                 'competitor4_rate':'cr4', 
                 'competitor4_has_availability':'cha4',
                 'competitor4_price_percent_diff':'cppf4', 
                 'competitor5_rate':'cr5',
                 'competitor5_has_availability':'cha5', 
                 'competitor5_price_percent_diff':'cppf5',
                 'competitor6_rate':'cr6', 
                 'competitor6_has_availability':'cha6',
                 'competitor6_price_percent_diff':'cppf6', 
                 'competitor7_rate':'cr7',
                 'competitor7_has_availability':'cha7', 
                 'competitor7_price_percent_diff':'cppf7',
                 'competitor8_rate':'cr8', 
                 'competitor8_has_availability':'cha8',
                 'competitor8_price_percent_diff':'cppf8'})

In [ ]:
cols_competitive = ['cr1', 'cr2', 'cr3', 'cr4', 'cr5', 'cr6', 'cr7', 'cr8',
                    'cha1', 'cha2', 'cha3', 'cha4', 'cha5', 'cha6', 'cha7', 'cha8', 
                    'cppf1', 'cppf2', 'cppf3', 'cppf4', 'cppf5', 'cppf6', 'cppf7', 'cppf8']

df_ = df_train[cols_competitive]

df_.isnull().sum().sort_values(ascending=False).iloc[0]/len(df_train)

In [ ]:
print('Competitor data has a lot of missing values. \
We can either combine them all to create a feature or\
 we can fill them. I prefer not to fill\
 missing values as the ratio of missing values is too much.')


In [ ]:
df_cr = df_train[['booked', 'cr1', 'cr2', 'cr3', 'cr4', 'cr5', 'cr6', 'cr7', 'cr8']]


#df_cr = df_cr.dropna(subset=['cr1', 'cr2', 'cr3', 'cr4', 'cr5', 'cr6', 'cr7', 'cr8'], how='all', axis=0)
df_cr['info'] = df_cr.apply(lambda x: 1 if x.count() - 1 > 1 else 0, axis=1)
df_cr

In [ ]:
df_cr.corr()

In [ ]:
# price difference, how it affects?
df_ = df_train[['booked', 'cppf1', 'cppf2', 'cppf3', 'cppf4', 'cppf5',
                'cppf6', 'cppf7', 'cppf8']]
df_.corr()

# Correlation

In [ ]:
list_corr = ['site_id', 'user_country_id',
       'user_hist_stars', 'user_hist_paid', 'listing_country_id', 'listing_id',
       'listing_stars', 'listing_review_score', 'is_brand', 'location_score1',
       'location_score2', 'log_historical_price', 'listing_position',
       'price_usd', 'has_promotion', 'destination_id', 'length_of_stay',
       'booking_window', 'num_adults', 'num_kids', 'num_rooms',
       'stay_on_saturday', 'log_click_proportion', 'distance_to_dest',
       'random_sort', 'cr1', 'cr2', 'cr3', 'cr4', 'cr5', 'cr6', 'cr7', 'cr8',
                    'cha1', 'cha2', 'cha3', 'cha4', 'cha5', 'cha6', 'cha7', 'cha8', 
                    'cppf1', 'cppf2', 'cppf3', 'cppf4', 'cppf5', 'cppf6', 'cppf7', 'cppf8',
       'clicked', 'booking_value', 'booked', 'times_sold',
       'search_year', 'search_month', 'search_dayofweek', 'search_dayofyear',
       'search_hour', 'search_quarter', 'stay_year', 'stay_month',
       'stay_dayofweek', 'stay_dayofyear', 'stay_hour', 'stay_quarter',
       'stay_start']

df_corr = df_train[list_corr].corr()



In [ ]:
indices = []
cols = []
corrs = []

for c in df_corr.columns:
    for i in df_corr.index:
        co = abs(df_corr[c].loc[i])
        if(c == i):
            continue
        if(co >= 0.85):
            if((c in cols) & (i in indices)):
                continue
                
            indices.append(i)
            cols.append(c)
            corrs.append(co)
            
df_ = pd.DataFrame()
df_['feat1'] = indices
df_['feat2'] = cols
df_['corr'] = corrs
df_.drop_duplicates(inplace=True)
df_
            

In [ ]:
# pd.set_option('display.max_rows', None)
df_corr['clicked'].sort_values(ascending=False)



In [ ]:
df_corr['booked'].sort_values(ascending=False)



In [ ]:
print('For now, I will discard competitors\' data.')

df_train = df_train.drop(cols_competitive, axis=1)


In [ ]:
print('Let\'s drop IDs: %s\n'%['search_id', 'listing_id', 'site_id',
                         'user_country_id', 'listing_country_id',
                          'destination_id'
                         ])
df_train = df_train.drop(['search_id', 'listing_id', 'site_id',
                         'user_country_id', 'listing_country_id',
                          'destination_id'
                         ], axis=1)


# Collinearity

In [ ]:
print('Shape of the train data: ', df_train.shape)

cols_time = ['search_year', 'search_month', 'search_dayofweek', 'search_dayofyear',
       'search_hour', 'search_quarter', 'stay_year', 'stay_month',
       'stay_dayofweek', 'stay_dayofyear', 'stay_hour', 'stay_quarter']

df_ = df_train[cols_time].drop_duplicates()

from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

df_vif = pd.DataFrame()

X_ = df_
df_vif['feats'] = X_.columns
df_vif['VIF_value'] = [vif(X_.values, i) for i in range(X_.shape[1])]
df_vif



In [ ]:
# drop some columns: unnecessary
cols_to_drop1 = ['search_year', 'search_month', 'search_quarter',
               'stay_year', 'stay_month', 'stay_hour', 
               'stay_quarter']

df_vif = pd.DataFrame()

X_ = df_.drop(cols_to_drop1, axis=1)

df_vif['feats'] = X_.columns
df_vif['VIF_value'] = [vif(X_.values, i) for i in range(X_.shape[1])]
df_vif

In [ ]:
cols_to_drop2 =['search_dayofyear']

df_vif = pd.DataFrame()

X_ = X_.drop(cols_to_drop2, axis=1)

df_vif['feats'] = X_.columns
df_vif['VIF_value'] = [vif(X_.values, i) for i in range(X_.shape[1])]
df_vif

In [ ]:
df_train.columns

In [ ]:
df_train = df_train.drop(['timestamp', 'listing_position',
                         'booking_window', 'stay_start',
                          'user_hist_stars', 'user_hist_paid',
                          'location_score2', 'log_click_proportion',
                          'booking_value', 'search_year', 
                          'search_month', 'search_quarter',
                           'stay_year', 'stay_month', 'stay_hour', 
                           'stay_quarter', 'search_dayofyear'
                         ], axis=1)


In [ ]:
df_train.isna().sum()

In [ ]:
df_train.head()

In [ ]:
df_train['clicked'] = df_train['clicked'].apply(lambda x: 1 if x== True else 0)
df_train['booked'] = df_train['booked'].apply(lambda x: 1 if x== True else 0)
df_train['stay_on_saturday'] = df_train['stay_on_saturday'].apply(lambda x: 1 if x== True else 0)
df_train['random_sort'] = df_train['random_sort'].apply(lambda x: 1 if x== True else 0)
df_train['is_brand'] = df_train['is_brand'].apply(lambda x: 1 if x== True else 0)


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

df_vif = pd.DataFrame()

X_ = df_train.drop(['booked'], axis=1)
df_vif['feats'] = X_.columns
df_vif['VIF_value'] = [vif(X_.values, i) for i in range(X_.shape[1])]
df_vif


In [ ]:
X_.plot.scatter('listing_stars', 'listing_review_score')

In [ ]:
cols_vif_qe10 = list(df_vif[df_vif['VIF_value'] >= 10]['feats'])
print('We have collinearity between the features. \
The most predictible features using other features:\n\
%s'%cols_vif_qe10)

In [ ]:
df_train.shape

# Prepare Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
pd.options.mode.chained_assignment = None 

def prep_data(df):
    
    ##########################
    # correct the types
    df['stay_on_saturday'] = df['stay_on_saturday'].astype('bool')
    df['random_sort'] = df['random_sort'].astype('bool')
    df['clicked'] = df['clicked'].astype('bool')
    df['booked'] = df['booked'].astype('bool')
    df['is_brand'] = df['is_brand'].astype('bool')
    df['timestamp'] = pd.to_datetime(df['timestamp'])


    ##########################
    # add date/time features
    df['search_dayofweek'] = pd.DatetimeIndex(df['timestamp']).dayofweek
    df['search_hour'] = pd.DatetimeIndex(df['timestamp']).hour
    
    ##########################
    # drop records where a user hasn't purchased anything but rated
    si = df[(df['user_hist_paid'].isna()) & (~df['user_hist_stars'].isna())]['search_id'].unique()
    df = df[~df['search_id'].isin(si)]
    
    ##########################
    # add feature: how many times a listing sold (could be done faster)
    df_times_sold = pd.DataFrame()
    t = []
    hotels = df['listing_id'].unique()
    for h in hotels:
        t.append(len(df[(df['listing_id'] == h) & (df['log_historical_price'] != 0)]['log_historical_price'].unique()))
    df_times_sold['hotel'] = hotels
    df_times_sold['times_sold'] = t
    df = df.merge(df_times_sold, how='left', left_on='listing_id', right_on='hotel').drop(['hotel'], axis=1)
    
    ##########################
    # Let's include the start date of the stay 
    df['stay_start'] = df['timestamp'] + df['length_of_stay'].apply(timedelta)
    df['stay_dayofweek'] = pd.DatetimeIndex(df['stay_start']).dayofweek
    df['stay_dayofyear'] = pd.DatetimeIndex(df['stay_start']).dayofyear  

    ##########################   
    # fillna
    
    # listing_review_score: average
    df_ = df[['listing_id', 'listing_review_score']].drop_duplicates()
    score_avg = df_['listing_review_score'].mean()
    df['listing_review_score'] = df['listing_review_score'].fillna(score_avg)

    # distance_to_dest: average
    df_ = df[['search_id', 'distance_to_dest']].drop_duplicates()
    dist_avg = df_['distance_to_dest'].mean()
    df['distance_to_dest'] = df['distance_to_dest'].fillna(dist_avg)

    ##########################
    # drop competitor data
    cols_competitive = ['competitor1_rate', 'competitor1_has_availability', 
                        'competitor1_price_percent_diff', 'competitor2_rate', 
                        'competitor2_has_availability', 'competitor2_price_percent_diff', 
                        'competitor3_rate', 'competitor3_has_availability', 
                        'competitor3_price_percent_diff', 'competitor4_rate', 
                        'competitor4_has_availability', 'competitor4_price_percent_diff', 
                        'competitor5_rate', 'competitor5_has_availability', 
                        'competitor5_price_percent_diff', 'competitor6_rate', 
                        'competitor6_has_availability', 'competitor6_price_percent_diff', 
                        'competitor7_rate', 'competitor7_has_availability', 
                        'competitor7_price_percent_diff', 'competitor8_rate', 
                        'competitor8_has_availability', 'competitor8_price_percent_diff']
    
    df.drop(cols_competitive, axis=1, inplace=True)
    
    ##########################
    # drop columns
    cols_to_drop = ['timestamp', 'site_id', 'search_id', 
                    'listing_id', 'listing_position',
                'user_country_id', 'listing_country_id',
                 'booking_window', 'destination_id',
                'stay_start', 'user_hist_stars', 'user_hist_paid', 'location_score2',
             'log_click_proportion', 'booking_value',
                   'location_score2']
    
    df.drop(cols_to_drop, axis=1, inplace=True)
    
    ##########################
    # True/False -> 1/0
    df['clicked'] = df['clicked'].apply(lambda x: 1 if x== True else 0)
    df['booked'] = df['booked'].apply(lambda x: 1 if x== True else 0)
    df['stay_on_saturday'] = df['stay_on_saturday'].apply(lambda x: 1 if x== True else 0)
    df['random_sort'] = df['random_sort'].apply(lambda x: 1 if x== True else 0)
    df['is_brand'] = dfn['is_brand'].apply(lambda x: 1 if x== True else 0)


    return df


df_train = pd.read_csv('train_small.csv')
df_train = prep_data(df_train)

print('done')


# save for backup
df_train_backup = df_train.copy()

# MODEL

In [ ]:
print('Shape of the data:', df_train.shape)

df_train.describe()

In [ ]:
# model data
df_train.columns

In [ ]:
from sklearn.model_selection import train_test_split

X = df_train.drop(['booked'], axis=1)
y = df_train['booked']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print('Dataset is split having %d datapoints in train set and\
 %d in test set.'%(len(X_train), len(X_test)))

In [ ]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))


In [ ]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))


In [ ]:
# K Nearest Neighbors

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

# EKSTRA

In [ ]:
# QUESTIONS-ANSWERS

In [ ]:
len(df_train[df_train['booked'] == 1])/len(df_train)

In [ ]:
df_raw = pd.read_csv('train_small.csv')

In [ ]:
# privacy/security issues
# user

df_ = df_raw[['user_country_id', 'user_hist_stars', 'user_hist_paid']].drop_duplicates()

df_uci = df_['user_country_id'].value_counts().to_frame()
print(df_uci)
list_uci = list(df_uci[df_uci['user_country_id'] == 1].index)


In [ ]:
df_uhs = df_['user_hist_stars'].value_counts().to_frame()
print(df_uhs)
list_uhs = list(df_uhs[df_uhs['user_hist_stars'] == 1].index)



In [ ]:
df_uhp = df_['user_hist_paid'].value_counts().to_frame()
print(df_uhp)
list_uhp = list(df_uhp[df_uhp['user_hist_paid'] == 1].index)


In [ ]:
df_raw[(df_raw['user_hist_paid'].isin(list_uhp)) & 
       (df_raw['user_hist_stars'].isin(list_uhs)) &
       (df_raw['user_country_id'].isin(list_uci))]


In [ ]:
df_raw[df_raw['user_hist_paid'].isin([364.00])]

In [ ]:
# listing

df_ = df_raw[['listing_country_id', 'listing_id', 'listing_stars',
             'listing_review_score', 'is_brand', 'location_score1',
             'location_score2', 'log_historical_price', 'price_usd',
             'destination_id', 'log_click_proportion']].drop_duplicates()
# also competitor data

df_uci = df_['user_country_id'].value_counts().to_frame()
print(df_uci)
list_uci = list(df_uci[df_uci['user_country_id'] == 1].index)



In [1]:
# create test data
import pandas as pd
df = pd.read_csv('train.csv')

In [ ]:
df.columns

In [ ]:
df = df[['prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
   'prop_location_score1', 'prop_log_historical_price',
   'price_usd', 'promotion_flag', 'srch_length_of_stay',
   'srch_booking_window', 'srch_adults_count', 
    'srch_children_count', 'srch_room_count',
    'srch_saturday_night_bool', 'orig_destination_distance',
    'date_time', 'booking_bool'
   ]]
cols_old = ['prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
   'prop_location_score1', 'prop_log_historical_price',
   'price_usd', 'promotion_flag', 'srch_length_of_stay',
   'srch_booking_window', 'srch_adults_count', 
    'srch_children_count', 'srch_room_count',
    'srch_saturday_night_bool', 'orig_destination_distance',
    'date_time', 'booking_bool'
   ]

In [ ]:
cols_new = ['listing_id', 'listing_stars', 'listing_review_score', 
                   'is_brand',
   'location_score1', 'log_historical_price',
   'price_usd', 'has_promotion', 'length_of_stay',
   'booking_window', 'num_adults', 
    'num_kids', 'num_rooms',
    'stay_on_saturday', 'distance_to_dest',
    'timestamp', 'booked']

In [ ]:
rename_dict = {}

for i, j in zip(cols_old, cols_new):
    rename_dict[i] = j

df.rename(columns=rename_dict, inplace=True)


In [ ]:
# 'search_dayofweek', 'search_hour', 
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['search_dayofweek'] = pd.DatetimeIndex(df['timestamp']).dayofweek
df['search_hour'] = pd.DatetimeIndex(df['timestamp']).hour
    

#'times_sold',
df_times_sold = pd.DataFrame()
t = []
hotels = df['listing_id'].unique()
for h in hotels:
    t.append(len(df[(df['listing_id'] == h) & (df['log_historical_price'] != 0)]['log_historical_price'].unique()))
df_times_sold['hotel'] = hotels
df_times_sold['times_sold'] = t
df = df.merge(df_times_sold, how='left', left_on='listing_id', right_on='hotel').drop(['hotel'], axis=1)

In [ ]:
# 'stay_dayofweek', 'stay_dayofyear'
from datetime import datetime, timedelta
df['stay_start'] = df['timestamp'] + df['length_of_stay'].apply(timedelta)
df['stay_dayofweek'] = pd.DatetimeIndex(df['stay_start']).dayofweek
df['stay_dayofyear'] = pd.DatetimeIndex(df['stay_start']).dayofyear  

In [ ]:
df.columns

In [ ]:
df[['listing_stars', 'listing_review_score', 
    'is_brand',
   'location_score1', 'log_historical_price',
   'price_usd', 'has_promotion', 'length_of_stay',
   'booking_window', 'num_adults', 
    'num_kids', 'num_rooms',
    'stay_on_saturday', 'distance_to_dest',
    'timestamp', 'booked']]